In [2]:
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import get_lodging_scores
import cv2  # importing cv
import imutils
import glob

#We need these in this file:
import load_read_name_extractor as lrne
import os

In [3]:
def it_name_extract_labels_from_img(names):
    plots = []
    temp = []
    for i in range(len(names)):
        name_t = os.path.basename(names[i])
        #name_t = name_t.replace(".tif", "")
        for j in range(1,len(name_t)):
            if (names[i][-j] == "_"):
                temp.append(name_t[-(j-1):len(name_t)].replace(".jpeg", ""))
                break
        temp.append(name_t.replace(temp[0], "").replace("_.jpeg", ""))
        plots.append(temp)
        temp = []
    return plots

In [4]:
features_load = np.load('/hog_fetures.npy', allow_pickle=True)
names_load = np.load('Features/harlick_martin_names.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

FileNotFoundError: [Errno 2] No such file or directory: 'hog_fetures.npy'

In [37]:
labels = get_lodging_scores.get_labels()
#[ROI, Visual SCORE, FLIGHTFolder]
print(features_load[0])
ht_mean = features_load[0].mean(axis=0)
ht_mean



[ 0.35711781  0.59375268  0.91123245  3.35318089  0.97736329  2.20893906
 12.81897088  1.76712487  1.81390284  0.05170437  0.25892179 -0.85631657
  0.96628899]


1.9401679487174872

In [1]:
names = it_name_extract_labels_from_img(names_load)

NameError: name 'it_name_extract_labels_from_img' is not defined

In [39]:
def match_pic_label(features, labels, names):
    n = len(labels)
    m = len(names)
    data_frame = []
    for i in range(n):
        for j in range(m):
            if (str(labels[i][0]) == str(names[j][0]) and labels[i][2] == names[j][1]):
                data_frame.append(np.hstack((features[j], labels[i][1])))
    return data_frame

In [40]:
clf = svm.SVC(kernel="sigmoid")
features_aar = np.array(features_load)
data_frame = match_pic_label(features_load, labels, names)
print(data_frame[0])
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


[ 0.35711781  0.59375268  0.91123245  3.35318089  0.97736329  2.20893906
 12.81897088  1.76712487  1.81390284  0.05170437  0.25892179 -0.85631657
  0.96628899 50.        ]


In [41]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [42]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

[65. 50. 40. 15. 30. 70. 65. 10. 55. 45. 30. 20.]


SVC(kernel='sigmoid')

In [43]:
y_pred = clf.predict(x_test)

In [44]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.0


              precision    recall  f1-score   support

         3.0       0.00      0.00      0.00       1.0
        15.0       0.00      0.00      0.00       1.0
        30.0       0.00      0.00      0.00       0.0
        75.0       0.00      0.00      0.00       1.0

    accuracy                           0.00       3.0
   macro avg       0.00      0.00      0.00       3.0
weighted avg       0.00      0.00      0.00       3.0



c:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marti\anaconda3\lib\site-packages\sklearn\metrics\_cla